In [1]:
# original size 258866 # 258841 # 258816
import sys
import os
from torch.utils.data import DataLoader
sys.path.append("..")
from tqdm import tqdm
import numpy as np
import pandas as pd

from utils import train_test_split, create_metadata_df
from dataset_xy import CARLADatasetXY
import json
from send2trash import send2trash

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_data = os.path.join("..", "..", "data", "data")

config_xy = {"used_inputs": ["measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])

# Create Dataset & DataLoader
dataset = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data, config=config_xy)
dataloader = DataLoader(dataset=dataset, batch_size=64, num_workers=0, sampler=None, shuffle=False)

In [3]:
def measurements_to_df(dataloader):
    idxs, speed, steer, throttle, brake, command = [], [], [], [], [], []
    for idx, (x, y) in tqdm(enumerate(dataloader)):
        idxs.append(idx)
        speed += x['speed'].flatten().tolist()
        command += x['command'].flatten().tolist()
        steer += y['steer'].flatten().tolist()
        throttle += y['throttle'].flatten().tolist()
        brake += y['brake'].flatten().tolist()

    df_measurements = pd.DataFrame({"speed": speed, "command": command, "steer": steer, "throttle": throttle, "brake": brake}, index=list(range(len(speed))))
    df_measurements.to_pickle("measurements_.pickle")
    return df_measurements

def remove_entries_at_indices(idx_nan_list, df_meta_entire):
    count_files_deleted = 0
    for idx_nan in idx_nan_list:
        path_nan = os.path.join(df_meta_entire.loc[idx_nan]["dir"], "measurements", df_meta_entire.loc[idx_nan]["measurements"])
        with open(path_nan, 'r') as f:
            measurements = json.load(f)
        if not np.isnan(measurements["steer"]):
            print("Break: indexes do not fit!")
            break
        dir = df_meta_entire["dir"].loc[idx_nan]
        dirs_sensors = os.listdir(dir)
        number_entry = df_meta_entire["measurements"].loc[idx_nan].split(".")[0]
        for dir_sensor in dirs_sensors:
            if not dir_sensor.startswith("."):
                files = os.listdir(os.path.join(dir, dir_sensor))
                for file in files:
                    if file.startswith(number_entry):
                        count_files_deleted += 1
                        # os.remove(os.path.join(dir, dir_sensor, file))
                        send2trash(os.path.join(dir, dir_sensor, file))
    return count_files_deleted

def remove_steer_nan_entries(df_meta_entire, dataloader):
    df_measurements = measurements_to_df(dataloader)
    print("df_measurements was created")
    indices_nan = df_meta_data[df_measurements["steer"].isna()].index.tolist()
    remove_entries_at_indices(indices_nan, df_meta_entire)
    return indices_nan

In [4]:
indices_nan = remove_steer_nan_entries(df_meta_data, dataloader)

448it [00:14, 30.92it/s]

In [ ]:
# should be 25
len(indices_nan)